# Histology Viewer
This example shows you how to use Syd to create an interactive histology viewer. 
Syd is not at all a replacement for more powerful software like ImageJ, but it is a quick way to look at your data! This shows how to load a histology image of a mouse brain and view it interactively from each plane (coronal, transverse, and sagittal).

You need to run the second cell to download the data from the GitHub release. 

In [ ]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    pass

In [ ]:
import os
import urllib.request
import zipfile

def download_if_needed(url: str, local_path: str) -> None:
    """Downloads the file from the given URL if it doesn't already exist."""
    if not os.path.exists(local_path):
        print(f"Downloading {url} to {local_path}...")
        urllib.request.urlretrieve(url, local_path)
    else:
        print(f"Zip file already exists at {local_path}")

def extract_if_needed(local_path: str, extract_dir: str) -> None:
    """Extracts the zip file to the given directory if not already extracted."""
    # Check for any existing file in the zip that indicates extraction was done
    with zipfile.ZipFile(local_path, 'r') as zip_ref:
        # We'll check only the first file in the archive
        first_file = zip_ref.namelist()[0]
        first_file_path = os.path.join(extract_dir, first_file)
        if not os.path.exists(first_file_path):
            print(f"Extracting {local_path} to {extract_dir}...")
            zip_ref.extractall(extract_dir)
        else:
            print(f"Contents already extracted to {extract_dir}")

# Example usage
zip_url = "https://github.com/landoskape/syd/releases/download/mousebrain-data/mousebrain.zip"
local_path = "images.zip"
download_if_needed(zip_url, local_path)
extract_if_needed(local_path, ".")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tifffile
from syd import Viewer

class HistologyViewer(Viewer):
    planes: list[str] = ["coronal", "transverse", "sagittal"]

    def __init__(self):
        self.add_selection("plane", value=self.planes[0], options=self.planes)
        self.add_integer("slice", value=0, min=0, max=1)
        self.add_float("vmax", value=1.0, min=0, max=1.0, step=0.001)
        self.add_float("redgreen_balance", value=0.5, min=0, max=1.0)
        self.on_change("plane", self.update_slice_limits)
        self.setup_volume()

    def setup_volume(self):
        red = np.array(tifffile.imread("./mousebrain/histology-viewer-red.tif"))
        green = np.array(tifffile.imread("./mousebrain/histology-viewer-green.tif"))
        blue = np.zeros_like(red)
        self._current_volume = np.stack([red, green, blue], axis=-1)
        self.update_slice_limits(self.state)        

    def update_slice_limits(self, state):
        dim = self.planes.index(state["plane"])
        self.update_integer("slice", max=self._current_volume.shape[dim] - 1)

    def get_slice(self, state):
        slice = state["slice"]
        dim = self.planes.index(state["plane"])
        c_slice = np.take(self._current_volume, slice, axis=dim)
        if dim == 2:
            c_slice = np.transpose(c_slice, (1, 0, 2))
        if dim == 1:
            c_slice = c_slice[::-1]
        c_slice = c_slice / np.max(c_slice)
        c_slice[:, :, 0] = c_slice[:, :, 0] * state["redgreen_balance"]
        c_slice[:, :, 1] = c_slice[:, :, 1] * (1 - state["redgreen_balance"])
        return c_slice

    def plot(self, state):
        vmax = state["vmax"]
        c_slice = self.get_slice(state)
        c_slice = np.clip(c_slice / vmax, 0, 1)
        fig, ax = plt.subplots(1, 1)
        ax.imshow(c_slice, aspect="equal")
        return fig

viewer = HistologyViewer()
viewer.show()